In [0]:
dbutils.widgets.text("catalog_name","")
dbutils.widgets.text("schema_name","")

In [0]:
catalog_name=dbutils.widgets.get("catalog_name")
schema_name=dbutils.widgets.get("schema_name")

In [0]:
from pyspark.sql.functions import *

In [0]:
patient_df=spark.read.table(f"{catalog_name}.silver.patient_data").drop("source_file")
appointment_df=spark.read.table(f"{catalog_name}.silver.appointment_data").drop("source_file")
lab_result_df=spark.read.table(f"{catalog_name}.silver.lab_results_data").drop("source_file")
symptom_df=spark.read.table(f"{catalog_name}.silver.symptons_data").drop("source_file")
medical_history_df=spark.read.table(f"{catalog_name}.silver.medical_history_data").drop("source_file")
medication_df=spark.read.table(f"{catalog_name}.silver.medication_data").drop("source_file")

In [0]:
from pyspark.sql.functions import col

joined_df = (
    patient_df.alias("a")
    .join(appointment_df.alias("b"), "patient_id", "left")
    .select(
        "a.*",
        col("b.doctor_name").alias("appointment_doctor_name"),
        col("b.appointment_date").alias("appointment_date"),
        col("b.reason_for_visit").alias("reason_for_visit"),
        col("b.status").alias("status")
    )
    .join(lab_result_df.alias("c"), "patient_id", "left")
    .join(symptom_df.alias("d"), "patient_id", "left")
    .join(medical_history_df.alias("e"), "patient_id", "left")
    .join(medication_df.alias("f"), "patient_id", "left")
)

display(joined_df)

In [0]:
joined_df=joined_df.withColumn("merged_desc",concat_ws("|",*[col(i) for i in joined_df.columns]))

In [0]:
display(joined_df)

In [0]:
joined_df.write.mode('overwrite').option("overwriteSchema",True).saveAsTable(f'{catalog_name}.{schema_name}.master_dim_patient_info')